In [12]:
import numpy as np
import pandas as pd
import math
import random
import praw
from sklearn.preprocessing import LabelEncoder
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [13]:
le = LabelEncoder()
tokenizer = RegexpTokenizer(r'\w+')
cv = CountVectorizer(ngram_range=(1,2))
mn  = MultinomialNB()
port = PorterStemmer()

In [14]:
def cleanText(text):
  t0 = text.lower()
  t1 = tokenizer.tokenize(t0)
  t2 = [t for t in t1 if t not in stopwords.words('english')]
  t3 = [port.stem(t) for t in t2]
  t4 = " ".join(t3)
  return t4
def redScrape(num):
    user_agent = "Scraper 1.0"
    reddit = praw.Reddit(
        client_id = "CCR9qratPsoFxW6lUtLsiA",
        client_secret = "DrlkhN7yc9u0tgj1caxThBcNzjB-sA",
        user_agent=user_agent
    )
    text = []
    headlines = set()
    posts = reddit.subreddit('politics').hot(limit=num)
    for post in posts:
        # print(post.author)
        # print(post.title)
        text.append(post.title)
        # print(post.selftext)
    return text

In [19]:
def sentimentAnalyse(path,trainingLimits,postLimit):
  data = pd.read_csv(path)
  data.columns = ["Num","Topic","Class","Sentence"]
  data = data.dropna()
  X = data.iloc[:trainingLimits,3]
  Y = data.iloc[:trainingLimits,2]  
  le.fit(Y)
  Y = le.transform(Y)
  X_clean = [cleanText(t) for t in X]
  X_clean = pd.DataFrame(X_clean)
  #Irrelevant - 0
  #Negative - 1
  #Neutral - 2
  #Positive - 3
  topic = np.full(X.shape[0],-1)
  X_clean['topic'] = topic
  for i in range(X_clean.shape[0]):
    res = X_clean.iloc[i,0].split(" ")
    for r in res:
      if r == "come":
        X_clean.iloc[i,1] = 0
      if r == "Crime":
        X_clean.iloc[i,1] = 1
      if r == "Transport":
        X_clean.iloc[i,1] = 2
      if r == "Health":
        X_clean.iloc[i,1] = 3
      if r == "Hygiene":
        X_clean.iloc[i,1] = 4
  X_vec = cv.fit_transform(X_clean[0]).toarray()
  mn.fit(X_vec,Y)
  text = redScrape(postLimit)
  text_clean = [cleanText(t) for t in X]
  text_clean = pd.DataFrame(text_clean)
  topic = np.full(text_clean.shape[0],-1)
  text_clean['topic'] = topic
  for i in range(text_clean.shape[0]):
    res = text_clean.iloc[i,0].split(" ")
    for r in res:
      if r == "Education":
        text_clean.iloc[i,1] = 0
      if r == "Crime":
        text_clean.iloc[i,1] = 1
      if r == "Transport":
        text_clean.iloc[i,1] = 2
      if r == "Health":
        text_clean.iloc[i,1] = 3
      if r == "Hygiene":
        text_clean.iloc[i,1] = 4
  text_vec = cv.fit_transform(text_clean[0]).toarray()
  Y_pred = mn.predict(text_vec)
  op = np.full((Y_pred.shape[0],5),0)
  for i in range(Y_pred.shape[0]):
    if X_clean.iloc[i,1] == 0:
      op[i,0] = Y_pred[i]
    if X_clean.iloc[i,1] == 1:
      op[i,1] = Y_pred[i]
    if X_clean.iloc[i,1] == 2:
      op[i,2] = Y_pred[i]
    if X_clean.iloc[i,1] == 3:
      op[i,3] = Y_pred[i]
    if X_clean.iloc[i,1] == 4:
      op[i,4] = Y_pred[i]
  return op

In [ ]:
sentimentAnalyse("/home/eshan/Eshan/Code Linux/twitterSentAnalysis/twitter_training.csv",100,100)